In [1]:
import pdfplumber
import re

In [2]:
pdf_file_path='C:/Users/sriyo/Desktop/jets/NLP Bot/files/NCERT Biology.pdf'

def extract_text_with_spaces(file_path):
    extracted_text = ""

    with pdfplumber.open(file_path) as pdf:
        num_pages = len(pdf.pages)
        for page_num in range(num_pages):
            page = pdf.pages[page_num]
            text = page.extract_text()
            extracted_text += f"Page {page_num + 1}:\n{text}\n"
    return extracted_text

pdf_text = extract_text_with_spaces(pdf_file_path)
print(pdf_text)

Page 1:
CHAPTER 2
HUMAN REPRODUCTION
2.1 The Male Reproductive
System
As you are aware, humans are sexually reproducing and
2.2 The Female Reproductive
viviparous. The reproductive events in humans include
System
formation of gametes (gametogenesis), i.e., sperms in males
2.3 Gametogenesis
and ovum in females, transfer of sperms into the female
2.4 Menstrual Cycle genital tract (insemination) and fusion of male and female
gametes (fertilisation) leading to formation of zygote. This
2.5 Fertilisation and
is followed by formation and development of blastocyst
Implantation
and its attachment to the uterine wall (implantation),
2.6 Pregnancy and Embryonic embryonic development (gestation) and delivery of the
Development baby (parturition). You have learnt that these reproductive
events occur after puberty. There are remarkable
2.7 Parturition and Lactation
differences between the reproductive events in the male
and in the female, for example, sperm formation continues
even in old men, but 

In [9]:
import pdfplumber
import re

def preprocess_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z0-9\s.]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    return text

def extract_and_preprocess_text(file_path):
    preprocessed_texts = [] 
    with pdfplumber.open(file_path) as pdf:
        num_pages = len(pdf.pages)

        for page_num in range(num_pages):
            page = pdf.pages[page_num]
            text = page.extract_text()
            preprocessed_text = preprocess_text(text)
            preprocessed_texts.append(preprocessed_text)

            print(f"Page {page_num + 1}:\n{preprocessed_text}\n")

    return preprocessed_texts 


pdf_file_path = 'C:/Users/sriyo/Desktop/jets/NLP Bot/files/NCERT Biology.pdf'
preprocessed_texts = extract_and_preprocess_text(pdf_file_path)



In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")
all_Sentence=[]
def tokenize_into_sentences(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences


for preprocessed_text in preprocessed_texts:
    sentences = tokenize_into_sentences(preprocessed_text)
    print(sentences)
    all_Sentence.append(sentences)



['chapter 2 human reproduction 2.1 the male reproductive system as you are aware humans are sexually reproducing and 2.2 the female reproductive viviparous.', 'the reproductive events in humans include system formation of gametes gametogenesis i.e. sperms in males 2.3 gametogenesis and ovum in females transfer of sperms into the female 2.4 menstrual cycle genital tract insemination and fusion of male and female gametes fertilisation leading to formation of zygote.', 'this 2.5 fertilisation and is followed by formation and development of blastocyst implantation and its attachment to the uterine wall implantation 2.6 pregnancy and embryonic embryonic development gestation and delivery of the development baby parturition.', 'you have learnt that these reproductive events occur after puberty.', 'there are remarkable 2.7 parturition and lactation differences between the reproductive events in the male and in the female for example sperm formation continues even in old men but formation of o

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re

def generate_questions_for_list(sentences, model, tokenizer, top_k=50, top_p=0.95):
    generated_questions=set()
    for sentence in sentences:
        input_text_with_prefix=f"Generate a question for the following text: {sentence}"
        inputs=tokenizer(input_text_with_prefix, return_tensors="pt")

        with torch.no_grad():
            outputs=model.generate(
                **inputs,
                max_length=50, 
                top_k=top_k,
                top_p=top_p,
                temperature=1.0,
            )

        generated_question=tokenizer.batch_decode(outputs, skip_special_tokens=True)
        split_questions=[question.strip()+'?' for question in re.split(r'[.!?]',generated_question[0])]
        generated_questions.update(split_questions)
    return generated_questions

tokenizer=AutoTokenizer.from_pretrained("wiselinjayajos/t5-end2end-questions-generation")
model=AutoModelForSeq2SeqLM.from_pretrained("wiselinjayajos/t5-end2end-questions-generation")

for sentences in all_Sentence:
    questions=generate_questions_for_list(sentences, model, tokenizer, top_k=50, top_p=0.95)
    print("Generated Questions:")
    for question in questions:
        print(question)


c:\Users\sriyo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


What is the male reproductive system as you know humans are sexually reproducing?
What is the female reproductive viviparous?
?
What is the reproductive event in humans?
What is the reproductive event in humans?
What is the reproductive event in humans?
What is the reproductive event in humans?
What is the reproductive event in humans?
?
What is the term for the formation and development of blastocyst implantation?
What is the term for the attachment of blastocyst to the uterine wall implantation?
What is the term for?
What happens after puberty?
What is the name of the event that occurs after puberty?
What is the name of the event that occurs after puberty?
?
What are the differences between the reproductive events in the male and female?
What does sperm formation continue even in old men?
What does formation of ovum cease in women around fifty years?
?
What are the two reproductive systems of human?
What is the question that we should examine?
?
What is the answer to the question for

In [6]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("en_core_web_sm")

def filter_single_question_by_context_and_grammar(input_sentence, generated_question, threshold=0.5):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([input_sentence, generated_question])
    similarity = cosine_similarity(vectors)[0][1]

    if similarity > threshold:
        # Check for proper grammar using spaCy
        if is_valid_question_grammar(generated_question):
            return generated_question
        else:
            return None
    else:
        return None

def is_valid_question_grammar(question):
    doc = nlp(question)

    # Check for at least one noun and one verb
    has_noun = any(token.pos_ == 'NOUN' for token in doc)
    has_verb = any(token.pos_ == 'VERB' for token in doc)

    return has_noun and has_verb

# Example usage
input_sentence = "This is the input sentence."
generated_question = "What is the main idea?"

if is_valid_question_grammar(generated_question):
    print("The generated question has valid grammar.")
else:
    print("The generated question does not have valid grammar.")

# Example usage
input_sentence = "This is the input sentence."
generated_question = "What is the main idea?"

filtered_question = filter_single_question_by_context_and_grammar(input_sentence, generated_question, threshold=0.5)

if filtered_question:
    print("Filtered Question:", filtered_question)
else:
    print("Generated question is not relevant or does not have valid grammar.")


The generated question does not have valid grammar.
Generated question is not relevant or does not have valid grammar.
